# 사진 준비하기

1. 얼굴이 포함된 사진을 준비하고
2. 사진으로부터 얼굴 영역 face landmark 를 찾아냅니다. (landmark를 찾기 위해서는 얼굴의 bounding box를 먼저 찾아야합니다.)
3. 찾아진 영역으로 부터 머리에 왕관 스티커를 붙여넣겠습니다.

In [1]:
#!pip install opencv-python

In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import dlib

In [4]:
img_bgr = cv2.imread('../../data/img/E10/image.jpeg')
img_bgr = cv2.resize(img_bgr, (640, 360))
img_show = img_bgr.copy() # 출력용 이미지 복사

In [ ]:
# !pip install cmake
# !pip install dlib # dlib 패키지 설치

# 